# **Obtención y preparación de datos**

# OD18. Funciones de Mapeo

pandas ofrece varios métodos para aplicar funciones a los valores de una serie o de un dataframe, o para sustituir dichos valores por otros aplicando un cierto "mapeado". Más concretamente nos encontramos con los siguientes métodos:

* **pandas.Series.apply**: aplica una función a cada uno de los elementos de la serie cuyo resultado, por lo tanto, tendrá el mismo tamaño que la serie original.
* **pandas.Series.map**: devuelve una serie del mismo tamaño que la original en la que cada valor ha sido sustituido por otro valor resultante de aplicar una "función de mapeado".
* **pandas.DataFrame.applymap**: aplica una función a cada uno de los elementos del dataframe que, por lo tanto, tendrá el mismo tamaño que el dataframe original.
* **pandas.DataFrame.apply**: aplica una función a las filas o a las columnas de un dataframe. Si, por ejemplo, se aplica a las filas, el resultado será una serie con tantos valores como filas tuviese el dataframe original.

Los nombres pueden parecer un tanto confusos: uno podría esperar que el método apply tuviese el mismo comportamiento en series y en dataframes y, en realidad, el método equivalente al método apply de las series es el applymap de los dataframes.

In [1]:
import numpy as np
import pandas as pd

## <font color='blue'>**El método Series.apply**</font>

El método **pandas.Series.apply** permite aplicar a cada uno de los elementos de la serie una función. Ésta deberá aceptar un único valor como argumento y devolver también un único valor.

In [2]:
s = pd.Series([2, 5, 4])
s

0    2
1    5
2    4
dtype: int64

Una función que eleve al cubo el argumento de entrada:



In [3]:
def cubo(n):
  return n ** 3

Podemos usar el método **apply** para aplicar esta función a cada uno de los elementos de la serie $s$:



In [4]:
s.apply(cubo) #aplica la función a la Serie s 

0      8
1    125
2     64
dtype: int64

El resultado es también una serie pandas.

## <font color='blue'>**El método Series.map**</font>

El método **pandas.Series.map** aplicado a una serie $s$ acepta un argumento que indica el tipo de mapeo a realizar y devuelve una serie equivalente a $s$ con sus valores una vez mapeados. Por ejemplo, supongamos que tenemos una serie cuyos valores representan el mes en el que se ha realizado una venta.

In [5]:
ventas = pd.Series([1, 2, 1, 1, 3, 1])
ventas

0    1
1    2
2    1
3    1
4    3
5    1
dtype: int64

Y supongamos que queremos generar una serie equivalente a ésta en la que cada mes aparezca representado por su nombre, y no por un número.

### <font color='blue'>**Uso de un diccionario como función de mapeo**</font>

Una de las formas que tenemos de definir este "mapeo" entre números y cadenas de texto es utilizando un diccionario:

In [6]:
meses = {1: "ene", 2: "feb", 3: "mar"}
meses

{1: 'ene', 2: 'feb', 3: 'mar'}

Ahora, si ejecutamos el método **map** añadiendo como argumento este diccionario, se devolverá la serie que buscábamos:



In [7]:
ventas.map(meses) #Map reclasifica la información, usando map para reemplazar los valores por los key de meses (ojo que meses es un diccionario)

0    ene
1    feb
2    ene
3    ene
4    mar
5    ene
dtype: object

### <font color='blue'>**Uso de una serie como función de mapeo**</font>

El método también admite como "función de mapeo" una serie:

In [8]:
meses = pd.Series(["ene", "feb", "mar"], index = [1, 2, 3]) #aquí meses es una Serie con los index como key de cruce
meses

1    ene
2    feb
3    mar
dtype: object

In [9]:
ventas.map(meses) #meses es la reclasificación de la columna, usando como cruce el valor de la Serie en ventas y el index en meses

0    ene
1    feb
2    ene
3    ene
4    mar
5    ene
dtype: object

En este caso, cada valor de la serie original (ventas, en nuestro ejemplo) se mapeará con el elemento cuya etiqueta coincida con él.

### <font color='blue'>**Uso de una función como función de mapeo**</font>

El tercer método al que podemos recurrir es utilizar una función que acepte como entradas los valores que se encuentren en la serie original y devuelva el resultado del mapeo. Por ejemplo:


In [10]:
def mes_str(n):
  if n == 1:
    return "ene"
  elif n == 2:
    return "feb"
  elif n == 3:
    return "mar"

In [11]:
ventas.map(mes_str) #map también acepta una función, donde de acuerdo al input retorna un output

0    ene
1    feb
2    ene
3    ene
4    mar
5    ene
dtype: object

## <font color='blue'>**El método DataFrame.apply**</font>

Los dataframes tienen un método con el mismo nombre que el método **apply** de las series, **pandas.DataFrame.apply**, pero con funcionalidad diferente pues, en el caso de los dataframes, se aplica a lo largo de un eje del dataframe. Esto quiere decir que el argumento de entrada de la función a utilizar no va a ser un simple escalar, sino una serie cuyo índice va a ser el índice de filas del dataframe (si la función se aplica al eje 0) o el índice de columnas del dataframe (si la función se aplica al eje 1). El resultado del método también será una serie que estará formada por los valores calculados.

Por ejemplo, si tenemos el siguiente dataframe con las ventas de los productos A, B, C y D a lo largo de los meses de enero, febrero y marzo:

In [12]:
ventas = pd.DataFrame({"A": [3, 3, 1],
                   "B": [1, 5, 2],
                   "C": [3, 7, 2],
                   "D": [7, 2, 3]},
                  index = ["ene", "feb", "mar"])
ventas

,A,B,C,D
ene,3,1,3,7
feb,3,5,7,2
mar,1,2,2,3


Podríamos estar interesados en calcular el rango en el que se mueven las ventas, es decir, la diferencia entre el mayor y el menor valor de ventas. Para ello, sabiendo que dicho rango se va a aplicar a una fila o a una columna -es decir, a una serie-, definimos la siguiente función:

In [13]:
def rango(s): #definir función antes de insertar
  return max(s) - min(s)

Esta función acepta un iterable y devuelve la diferencia entre el valor máximo y el mínimo.

Ahora podemos aplicar esta función a nuestro dataframe de ventas. Por defecto se va a aplicar al eje 0 (eje vertical):

In [14]:
ventas.apply(rango) #Solo queda la Serie resultante de la función

A    2
B    4
C    5
D    5
dtype: int64

Si nos fijamos en la columna A, el valor máximo es 3 y el mínimo es 1, de forma que su diferencia es 2, tal y como se muestra en el resultado del método **apply**.

Si aplicamos el método a lo largo del eje 1 (eje horizontal), obtendremos la diferencia entre el mayor y el menor valor de cada fila:

In [15]:
ventas.apply(rango, axis = 1) #si queremos aplicar la función a las filas, señalar axis=1 (axis=0 por defecto)

ene    6
feb    5
mar    2
dtype: int64

## <font color='blue'>**El método DataFrame.applymap**</font>

Al contrario de lo que ocurría con el método **apply** de los dataframes, el método **pandas.DataFrame.applymap** aplica una función que acepta y devuelve un único escalar, función que se va a aplicar a todos los elementos del dataframe.*texto en cursiva*

In [16]:
ventas = pd.DataFrame({"A": [3, 3, 1],
                   "B": [1, 5, 2],
                   "C": [3, 7, 2],
                   "D": [7, 2, 3]},
                  index = ["ene", "feb", "mar"])
ventas

,A,B,C,D
ene,3,1,3,7
feb,3,5,7,2
mar,1,2,2,3


Supongamos que queremos saber si los valores son pares o no. Para ello definimos una función que acepta un valor de entrada y devuelve el booleano True si el valor es par y False en caso contrario:

In [17]:
def par(n):
  if n/2 == n//2:
    return True
  else:
    return False

Ahora podemos aplicar el método añadiendo como argumento esta función:

In [18]:
ventas.applymap(par) #aplica para todos los elementos del DataFrame

,A,B,C,D
ene,False,False,False,False
feb,False,False,False,True
mar,False,True,True,False


Comprobamos que el resultado es un dataframe del mismo tamaño que el dataframe original en el que cada valor se ha sustituido por el resultado de aplicar la función indicada.

<font color="red">se observa que estas funciones son útiles para agregar columnas en un DataFrame que dependen de la agrupación de otros campos (especie de pivot_table interna).</font>